<a href="https://colab.research.google.com/github/Momilijaz96/VGG16_CIFAR100_Optim/blob/main/VGG16_CIFAR100(SGD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import models,transforms,datasets

%load_ext autotime

time: 573 µs (started: 2021-09-04 04:15:07 +00:00)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

time: 45.5 ms (started: 2021-09-04 04:15:07 +00:00)


# Import CIFAR-100 and Resize

In [ ]:
train_transform = transforms.Compose([transforms.Resize(64),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.RandomRotation(15),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])

test_transform = transforms.Compose([transforms.Resize(64),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.48,0.4593,0.4155),(0.2774,0.2794,0.2794))])

time: 5.49 ms (started: 2021-09-04 04:15:07 +00:00)


In [ ]:
train_set = datasets.CIFAR100(root = "CIFAR100", train = True, download = True, transform = train_transform)

Files already downloaded and verified
time: 1.07 s (started: 2021-09-04 04:15:07 +00:00)


In [ ]:
test_set = datasets.CIFAR100(root = "CIFAR100", train = False, download = True, transform = test_transform)

Files already downloaded and verified
time: 803 ms (started: 2021-09-04 04:15:08 +00:00)


In [ ]:
train_set[0][0].shape

torch.Size([3, 64, 64])

time: 8.68 ms (started: 2021-09-04 04:15:09 +00:00)


In [ ]:
test_set[0][0].shape

torch.Size([3, 64, 64])

time: 6.56 ms (started: 2021-09-04 04:15:09 +00:00)


# VGG16

In [ ]:
# Number of classes in the dataset
num_classes = 100

# Batch size for training
batch_size = 64

# Number of epochs
num_epochs = 20

time: 4.75 ms (started: 2021-09-04 04:15:09 +00:00)


In [ ]:
vgg16_model = models.vgg16(pretrained = True) #130million+ parameters

#set_parameter_requires_grad for classifier layers
'''
for param in vgg16_model.parameters():
  param.requires_grad=False
'''

#Modify last layer out classes
vgg16_model.classifier[6].out_features = num_classes

vgg16_model = vgg16_model.to(device)

time: 4 s (started: 2021-09-04 04:15:09 +00:00)


In [ ]:
print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [ ]:
criterion = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(vgg16_model.parameters(), lr = 0.001, momentum = 0.9)
lr_scheduler = optim.lr_scheduler.StepLR(sgd_optimizer, step_size = 10, gamma = 0.1)

train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_set, batch_size = batch_size, shuffle = True)

time: 10.8 ms (started: 2021-09-04 04:15:13 +00:00)


In [ ]:
def fit(model, dataset, optimizer, scheduler, criterion):
  #Set model to training mode
  model.train()
  scheduler.step()
  #Iterate over data
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    #Reset the gradients
    optimizer.zero_grad()
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

time: 8.18 ms (started: 2021-09-04 04:15:13 +00:00)


In [ ]:
def validate(model, dataset, criterion):
  #set model to evaluation mode
  model.eval()
  acc = 0
  loss = []
  for data, targets in dataset:
    data = data.to(device)
    targets = targets.to(device)
    out = model(data)
    #Get loss
    l = criterion(out, targets)
    loss.append(l.item())
    #Get index of class label
    _,preds = torch.max(out.data,1)
    #Get accuracy
    acc += torch.sum(preds == targets).item()

  return 100*acc/len(dataset.dataset), np.mean(np.array(loss))

time: 18.4 ms (started: 2021-09-04 04:15:13 +00:00)


In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []

#Loop for num_epochs
for epoch in range(num_epochs):
  #Fit model
  fit(vgg16_model, train_dataloader, sgd_optimizer, lr_scheduler, criterion)

  #Train validation
  trn_acc, trn_lss = validate(vgg16_model, train_dataloader, criterion)
  train_loss.append(trn_lss)
  train_acc.append(trn_acc)

  #Test validation
  tst_acc, tst_lss = validate(vgg16_model, test_dataloader, criterion)
  test_loss.append(tst_lss)
  test_acc.append(tst_acc)

  print(f'Epoch:{epoch+1}\t Train_Loss:{train_loss[-1]:.4f}\t Train_Acc:{train_acc[-1]:.4f}\t Test_Loss:{test_loss[-1]:.4f}\t Test_Acc:{test_acc[-1]:.4f}')


Epoch:1	 Train_Loss:2.9522	 Train_Acc:23.3820	 Test_Loss:2.9883	 Test_Acc:21.6800
Epoch:2	 Train_Loss:1.9591	 Train_Acc:45.3800	 Test_Loss:1.9788	 Test_Acc:44.4700
Epoch:3	 Train_Loss:1.5607	 Train_Acc:55.6080	 Test_Loss:1.6354	 Test_Acc:53.0100
Epoch:4	 Train_Loss:1.3547	 Train_Acc:60.5880	 Test_Loss:1.4688	 Test_Acc:57.5600
Epoch:5	 Train_Loss:1.2058	 Train_Acc:64.6740	 Test_Loss:1.3700	 Test_Acc:60.7000
Epoch:6	 Train_Loss:1.0951	 Train_Acc:67.4580	 Test_Loss:1.2949	 Test_Acc:62.5500
Epoch:7	 Train_Loss:1.0223	 Train_Acc:70.0160	 Test_Loss:1.2805	 Test_Acc:63.3500
Epoch:8	 Train_Loss:0.8719	 Train_Acc:73.8780	 Test_Loss:1.1986	 Test_Acc:65.9000
Epoch:9	 Train_Loss:0.7510	 Train_Acc:77.3980	 Test_Loss:1.1566	 Test_Acc:67.1100
Epoch:10	 Train_Loss:0.6284	 Train_Acc:80.7140	 Test_Loss:1.0602	 Test_Acc:69.7700
Epoch:11	 Train_Loss:0.6068	 Train_Acc:81.3600	 Test_Loss:1.0614	 Test_Acc:69.9200
Epoch:12	 Train_Loss:0.5862	 Train_Acc:81.9420	 Test_Loss:1.0533	 Test_Acc:70.3100
Epoch:13	 Tra

# Save Model

In [ ]:
torch.save(vgg16_model.state_dict(), 'VGG16(SGD)')

time: 1.96 s (started: 2021-09-04 06:02:29 +00:00)
